<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-exploration" data-toc-modified-id="Data-exploration-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data exploration</a></span></li><li><span><a href="#XGBoost-model" data-toc-modified-id="XGBoost-model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>XGBoost model</a></span><ul class="toc-item"><li><span><a href="#Hyperparameter-tuning-with-Hyperopt" data-toc-modified-id="Hyperparameter-tuning-with-Hyperopt-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Hyperparameter tuning with Hyperopt</a></span></li><li><span><a href="#Model-fitting" data-toc-modified-id="Model-fitting-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Model fitting</a></span></li></ul></li><li><span><a href="#Saving-the-model" data-toc-modified-id="Saving-the-model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Saving the model</a></span></li><li><span><a href="#SHAP-values" data-toc-modified-id="SHAP-values-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>SHAP values</a></span></li><li><span><a href="#Fitting-additional-logistic-regression" data-toc-modified-id="Fitting-additional-logistic-regression-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Fitting additional logistic regression</a></span></li><li><span><a href="#Validation-check" data-toc-modified-id="Validation-check-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Validation check</a></span><ul class="toc-item"><li><span><a href="#Submission-XGB-SCORE" data-toc-modified-id="Submission-XGB-SCORE-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Submission XGB SCORE</a></span></li><li><span><a href="#Submission-combined-strategy" data-toc-modified-id="Submission-combined-strategy-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Submission combined strategy</a></span></li></ul></li><li><span><a href="#Misc" data-toc-modified-id="Misc-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Misc</a></span></li></ul></div>

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import random
import mofr
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier, plot_tree
from lightgbm import LGBMClassifier, plot_tree #LGBMRanker
import math
import pickle
import statsmodels.api as sm


import xgboost as xgb

In [3]:
seed=1234

In [4]:
# file = open("type2id.pkl",'rb')
# object_file = pickle.load(file)

In [5]:
file_name = "otto_xgb_model_13_cv.pkl"

In [6]:
df=pd.read_parquet('train_sample_full_cv.parquet')#train_sample_full_cv_hidden_targets_float
valid=pd.read_parquet('valid_sample_full_cv.parquet')

In [7]:
# df=pd.concat([df,valid])

In [8]:
df.columns=[x.lower() for x in df.columns]
valid.columns=[x.lower() for x in valid.columns]

In [9]:
# sessions_with_orders=pd.DataFrame(df.groupby('session_id')['target'].sum()).reset_index()
# sessions_with_orders=sessions_with_orders[sessions_with_orders['target']>0]
# df=df.merge(sessions_with_orders['session_id'], how='inner', on='session_id')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16016090 entries, 0 to 16016089
Data columns (total 40 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   target                          float64
 1   session_id                      float64
 2   aid                             float64
 3   in_session                      float64
 4   in_session_clicked              float64
 5   in_session_carted               float64
 6   in_session_ordered              float64
 7   in_session_score                float64
 8   in_session_score_time_weighted  float64
 9   times_in_top20                  float64
 10  times_in_top20_last_week        float64
 11  min_order_hours_from_click_avg  float64
 12  avg_order_hours_from_click_avg  float64
 13  avg_num_sessions_ordered        float64
 14  sum_num_sessions_ordered        float64
 15  avg_num_occured_overall         float64
 16  sum_num_occured_overall         float64
 17  last_num_sessions_ordered

In [11]:
df.head()

,target,session_id,aid,in_session,in_session_clicked,in_session_carted,in_session_ordered,in_session_score,in_session_score_time_weighted,times_in_top20,...,sess_hour,sess_dayofweek,sess_dayofmonth,number_carts_orders,number_carts,number_orders,last_seen,last_bought,bought_overall,seen_overall
0,0.0,9489516.0,233164.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,8.0,3.0,17.0,4.0,4.0,0.0,1.482124,60.349619,57.0,861.0
1,0.0,9489516.0,234450.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,8.0,3.0,17.0,4.0,4.0,0.0,2.253752,6.167958,547.0,3228.0
2,0.0,9489516.0,236091.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.0,3.0,17.0,4.0,4.0,0.0,8.632612,494.168694,1.0,11.0
3,0.0,9489516.0,238912.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.0,3.0,17.0,4.0,4.0,0.0,8.214399,8.214399,1.0,22.0
4,0.0,9489516.0,239613.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.0,3.0,17.0,4.0,4.0,0.0,8.308145,8.308145,8.0,39.0


In [12]:
df['null_flag']=(df['avg_order_hours_from_click_avg']<99999).apply(int)
df['in_session_carted_not_ordered']=((df['in_session_carted']>0)&(df['in_session_ordered']==0)).apply(int)
df['in_session_clicked_not_carted']=((df['in_session_clicked']>0)&(df['in_session_carted']==0)).apply(int)
valid['null_flag']=(valid['avg_order_hours_from_click_avg']<99999).apply(int)
valid['in_session_carted_not_ordered']=((valid['in_session_carted']>0)&(valid['in_session_ordered']==0)).apply(int)

In [13]:
df['null_flag'].value_counts(normalize=True)

1    0.752698
0    0.247302
Name: null_flag, dtype: float64

In [14]:
df['target'].value_counts()

0.0    15989660
1.0       26430
Name: target, dtype: int64

In [15]:
#df=df[df['null_flag']==1].reset_index()
#valid=valid[valid['null_flag']==1].reset_index()

In [16]:
df['target'].value_counts()

0.0    15989660
1.0       26430
Name: target, dtype: int64

In [17]:
rand=np.random.randint(1, 11, df.shape[0])

train_mask=(df['sess_days_till_test']+df['sess_duration']/24)>0#rand<=8
valid_mask=(df['sess_days_till_test']+df['sess_duration']/24)>0#rand>8

In [18]:
df[train_mask]['target'].value_counts()

0.0    15989660
1.0       26430
Name: target, dtype: int64

In [19]:
df[valid_mask]['target'].value_counts()

0.0    15989660
1.0       26430
Name: target, dtype: int64

In [20]:
col_target='target'
col_exclude=[
'session_id',
'aid',
# 'num_times_ordered_overall',
'number_carts_orders',
'number_carts',
'number_orders',
'sess_days_till_test',
'sess_dayofmonth',
# 'in_session',
# 'in_session_clicked',
# 'in_session_carted',
#  'in_session_ordered',
#  'in_session_score',

col_target,]

In [21]:
# df[col_target]=df[col_target].apply(int)

# Data exploration

In [22]:
col_preds=[col for col in df.columns[3:] if col not in col_exclude]

In [23]:
len(col_preds)

35

In [24]:
col_preds

['in_session',
 'in_session_clicked',
 'in_session_carted',
 'in_session_ordered',
 'in_session_score',
 'in_session_score_time_weighted',
 'times_in_top20',
 'times_in_top20_last_week',
 'min_order_hours_from_click_avg',
 'avg_order_hours_from_click_avg',
 'avg_num_sessions_ordered',
 'sum_num_sessions_ordered',
 'avg_num_occured_overall',
 'sum_num_occured_overall',
 'last_num_sessions_ordered',
 'last_hour_num_sessions_ordered',
 'num_times_ordered_overall',
 'sess_num_events',
 'sess_num_clicks',
 'sess_num_carts',
 'sess_num_orders',
 'sess_num_clicks_last_hour',
 'sess_num_carts_last_hour',
 'sess_num_orders_last_hour',
 'sess_duration',
 'sess_minute',
 'sess_hour',
 'sess_dayofweek',
 'last_seen',
 'last_bought',
 'bought_overall',
 'seen_overall',
 'null_flag',
 'in_session_carted_not_ordered',
 'in_session_clicked_not_carted']

# XGBoost model

## Hyperparameter tuning with Hyperopt

In [25]:
# from hyperopt import fmin, tpe, hp, anneal, Trials, STATUS_OK

# sample_train=df[train_mask].sample(250000)
# sample_valid=df[valid_mask].sample(100000)

# space={'max_depth': hp.quniform("max_depth", 3, 6, 1),
#         'gamma': hp.uniform ('gamma', 1,9),
#         'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
#         'reg_lambda' : hp.uniform('reg_lambda', 0,1),
#         'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
#         'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
#         'n_estimators': 100,
#         'seed': 0
#     }

# def objective(space):
#     clf=xgb.XGBClassifier(
#                     n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
#                     reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
#                     colsample_bytree=int(space['colsample_bytree']))
    
#     evaluation = [( sample_train[col_preds], sample_train[col_target]), ( sample_valid[col_preds], sample_valid[col_target])]
    
#     clf.fit(sample_train[col_preds], sample_train[col_target],
#             eval_set=evaluation, eval_metric="auc",
#             early_stopping_rounds=10,verbose=False)
    

#     pred = clf.predict_proba(sample_valid[col_preds])[:, 1]
#     gini = mofr.metrics.gini(sample_valid[col_target], pred)
#     print ("SCORE:", gini)
#     return {'loss': -gini, 'status': STATUS_OK }

# trials = Trials()

# best_hyperparams = fmin(fn = objective,
#                         space = space,
#                         algo = tpe.suggest,
#                         max_evals = 33,
#                         trials = trials)

# del sample_train
# del sample_valid

# hyperparams_name = "best_hyperparams_cv.pkl"

# # save
# pickle.dump(best_hyperparams, open(hyperparams_name, "wb"))

## Model fitting

In [26]:
hyperparams_name = "best_hyperparams_cv.pkl"
best_hyperparams= pickle.load(open(hyperparams_name, "rb"))

In [27]:
best_hyperparams['max_depth']=int(best_hyperparams['max_depth'])
best_hyperparams['max_depth']=4
best_hyperparams

{'colsample_bytree': 0.5378782340350697,
 'gamma': 3.938413440616859,
 'max_depth': 4,
 'min_child_weight': 6.0,
 'reg_alpha': 46.0,
 'reg_lambda': 0.4149847395970465}

In [28]:
train=df[train_mask]#&(df['null_flag']==1)
print(train['session_id'].max())
train[col_target].value_counts()

11098470.0


0.0    15989660
1.0       26430
Name: target, dtype: int64

In [ ]:
xgb_model = XGBClassifier(max_depth=4, seed=seed, colsample_bytree=0.5, gamma=1, min_child_weight=5, n_estimators=100)##**best_hyperparams
xgb_model.fit(train.loc[:, col_preds], train[col_target], verbose=0, eval_metric='auc')

In [ ]:
df['XGB_SCORE']=xgb_model.predict_proba(df[col_preds])[:, 1]
train['XGB_SCORE']=xgb_model.predict_proba(train[col_preds])[:, 1]
valid['XGB_SCORE']=xgb_model.predict_proba(valid[col_preds])[:, 1]

In [ ]:
print('The Lift on the train set is: '+ str(mofr.metrics.lift(train[col_target], train['XGB_SCORE'])))
print('The gini on the train set is: '+ str(mofr.metrics.gini(train[col_target].apply(int), train['XGB_SCORE'])))
# print('The accuracy on the train set is: '+ str(mofr.metrics.accuracy_score(train[col_target].apply(int), train['XGB_SCORE'].apply(lambda x: int(x>0.5)))))
print('\n')
print('The Lift on the valid set is: '+ str(mofr.metrics.lift(valid[col_target], valid['XGB_SCORE'])))
print('The gini on the valid set is: '+ str(mofr.metrics.gini(valid[col_target].apply(int), valid['XGB_SCORE'])))
# print('The accuracy on the valid set is: '+ str(mofr.metrics.accuracy_score(df[valid_mask][col_target].apply(int), df[valid_mask]['XGB_SCORE'].apply(lambda x: int(x>0.5)))))
print('\n')

In [ ]:
from mofr.basic_evaluators.ROCCurve import ROCCurveEvaluator
df['one']=1

rce=ROCCurveEvaluator()
rce.d(df[valid_mask]).t([(col_target,'one')]).s(['XGB_SCORE'])
rce.get_graph()

del df['one']

In [ ]:
sorted_idx = xgb_model.feature_importances_.argsort()
order_ = []
for i in sorted_idx:
  order_.append(xgb_model.feature_names_in_[i])
plt.figure(figsize=(10, 10))
fig = plt.barh(order_, xgb_model.feature_importances_[sorted_idx])
plt.xlabel("Xgboost Feature Importance")
plt.show()

In [ ]:
import os
os.environ["PATH"] += os.pathsep + 'c:/Program Files/Graphviz/bin/'

from xgboost import plot_tree
plt.rcParams["figure.figsize"] = (40,40)
plot_tree(xgb_model)
plt.show()

In [ ]:
# results=[]
# for col in col_preds:
#     results.append((col, np.abs(mofr.metrics.gini(df[valid_mask][col_target], df[valid_mask][col]))))
  
# pd.DataFrame(results, columns=['Predictor', 'GINI']).sort_values(by='GINI', ascending=False)[0:30]

# Saving the model

In [ ]:
#file_name = "otto_xgb_model_10_3weeks_no_nulls.pkl"

# save
pickle.dump(xgb_model, open(file_name, "wb"))

# # load
# #xgb_model= pickle.load(open(file_name, "rb"))

# SHAP values

In [ ]:
import shap  # package used to calculate Shap values
row_to_show = 1
data_for_prediction = train[col_preds].iloc[row_to_show]  # use 1 row of data here. Could use multiple rows if desired
data_for_prediction_array = data_for_prediction.values.reshape(1, -1)

# Create object that can calculate shap values
explainer = shap.TreeExplainer(xgb_model)

# Calculate Shap values
shap_values = explainer.shap_values(data_for_prediction_array)
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values, data_for_prediction)

In [ ]:
sample=train[col_preds].sample(100000)
shap_values = explainer.shap_values(sample)
shap.summary_plot(shap_values, sample[col_preds])

In [ ]:
sample=train.sample(100000)
results=[]
for col in col_preds:
    results.append((col, mofr.metrics.gini(sample[col_target], sample[col].fillna(-999))))

pd.DataFrame(results, columns=['Predictor', 'GINI']).sort_values(by='GINI', ascending=False)

In [ ]:
del sample

# Validation check

In [ ]:
# del df
# del train

In [ ]:
import pandas as pd
import numpy as np
import random
import mofr
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier, plot_tree
import math
import pickle


import xgboost as xgb

In [ ]:
#file_name = "otto_xgb_model_10_3weeks_no_nulls.pkl"

#load
xgb_model= pickle.load(open(file_name, "rb"))

In [ ]:
col_target='target'

In [ ]:
valid.head()

In [ ]:
valid[col_target].value_counts()

In [ ]:
coverage=(pd.DataFrame(df.groupby('session_id')[col_target].apply(np.sum)).reset_index()).merge(df.groupby('session_id')[['number_carts_orders','number_carts', 'number_orders']].apply(np.mean), how='inner', on='session_id')
target_coverage=np.sum(coverage[coverage['number_carts_orders']>0]['target'])/np.sum(coverage[coverage['number_carts_orders']>0]['number_carts_orders'].apply(lambda x: min([20,x])))
# target_avg_coverage=np.sum(coverage[coverage['number_carts_orders']>0]['target']/coverage[coverage['number_carts_orders']>0]['number_carts_orders'])
target_avg_coverage_carts=np.sum(coverage[coverage['number_carts']>0]['target'])/np.sum(coverage[coverage['number_carts']>0]['number_carts'].apply(lambda x: min([20,x])))
target_avg_coverage_orders=np.sum(coverage[coverage['number_orders']>0]['target'])/np.sum(coverage[coverage['number_orders']>0]['number_orders'].apply(lambda x: min([20,x])))

print(f'The coverage of the target on the train set is: {target_coverage}')
print(f'The mean coverage of carts on the train set is: {target_avg_coverage_carts}')
print(f'The mean coverage of orders on the train set is: {target_avg_coverage_orders}')
print(f'Max score possible with this set is: {0.1*0.5+0.3*target_avg_coverage_carts+0.6*target_avg_coverage_orders}')

In [ ]:
coverage=(pd.DataFrame(valid.groupby('session_id')[col_target].apply(np.sum)).reset_index()).merge(valid.groupby('session_id')[['number_carts_orders','number_carts', 'number_orders']].apply(np.mean), how='inner', on='session_id')
target_coverage=np.sum(coverage[coverage['number_carts_orders']>0]['target'])/np.sum(coverage[coverage['number_carts_orders']>0]['number_carts_orders'].apply(lambda x: min([20,x])))
# target_avg_coverage=np.sum(coverage[coverage['number_carts_orders']>0]['target']/coverage[coverage['number_carts_orders']>0]['number_carts_orders'])
target_avg_coverage_carts=np.sum(coverage[coverage['number_carts']>0]['target'])/np.sum(coverage[coverage['number_carts']>0]['number_carts'].apply(lambda x: min([20,x])))
target_avg_coverage_orders=np.sum(coverage[coverage['number_orders']>0]['target'])/np.sum(coverage[coverage['number_orders']>0]['number_orders'].apply(lambda x: min([20,x])))

print(f'The coverage of the target on the valid set is: {target_coverage}')
print(f'The mean coverage of carts on the valid set is: {target_avg_coverage_carts}')
print(f'The mean coverage of orders on the valid set is: {target_avg_coverage_orders}')
print(f'Max score possible with this set is: {0.1*0.5+0.3*target_avg_coverage_carts+0.6*target_avg_coverage_orders}')

## Submission XGB SCORE

In [ ]:
a=valid.groupby('session_id').apply(lambda x : x.sort_values(by = ['XGB_SCORE'], ascending = [False]).head(20).reset_index(drop = True)).reset_index(drop = True)
a['session_id']=a['session_id'].apply(lambda x: str(int(x)))
a['aid']=a['aid'].apply(lambda x: str(int(x)))
    
pred_df=pd.DataFrame(a.groupby('session_id')['aid'].apply(list).apply(' '.join)).reset_index(drop=False).rename(columns={'aid':'labels'})

In [ ]:
# a=valid.groupby('session_id').apply(lambda x : x.sort_values(by = ['in_session','times_in_top20','XGB_SCORE'], ascending = [False,False,False]).head(20).reset_index(drop = True)).reset_index(drop = True)
# a['session_id']=a['session_id'].apply(lambda x: str(int(x)))
# a['aid']=a['aid'].apply(lambda x: str(int(x)))
    
# pred_df=pd.DataFrame(a.groupby('session_id')['aid'].apply(list).apply(' '.join)).reset_index(drop=False).rename(columns={'aid':'labels'})

In [ ]:
# TOP CLICKS AND ORDERS IN TEST
top_clicks = ['485256', '1460571','108125','1551213','33343','613493','876493','152547','184976','1236775'
              ,'1406660','836852','331708','1531805','29735','554660','634452','986164','959208','832192']

top_carts = ['485256', '33343','613493','152547','876493','1406660','1736857','166037','1460571','1236775'
             ,'554660','660655','1531805','1022566','231487','122983','1562705','923948','332654','544144']

top_orders = ['876493', '1406660','1236775','166037','1460571','1531805','836852','634452','923948','1043508'
             ,'832192','258353','801774','332654','1596897','29735','544144','1581568','1462420','1006198']

In [ ]:
submission_clicks_valid=pred_df.copy()
submission_carts_valid=pred_df.copy()
submission_orders_valid=pred_df.copy()

submission_clicks_valid['session_type']=submission_clicks_valid['session_id'].apply(lambda x: x+'_clicks')
submission_carts_valid['session_type']=submission_carts_valid['session_id'].apply(lambda x: x+'_carts')
submission_orders_valid['session_type']=submission_orders_valid['session_id'].apply(lambda x: x+'_orders')     

In [ ]:
submission_clicks_valid['labels']=submission_clicks_valid['labels'].apply(lambda x: list(x.split(' ')) + top_clicks).apply(lambda x: ' '.join(list(dict.fromkeys(x))[0:20]))
submission_carts_valid['labels']=submission_carts_valid['labels'].apply(lambda x: list(x.split(' ')) + top_carts).apply(lambda x: ' '.join(list(dict.fromkeys(x))[0:20]))
submission_orders_valid['labels']=submission_orders_valid['labels'].apply(lambda x: list(x.split(' ')) + top_carts).apply(lambda x: ' '.join(list(dict.fromkeys(x))[0:20]))

In [ ]:
submission_final_valid=pd.concat([submission_clicks_valid[['session_type', 'labels']],submission_carts_valid[['session_type', 'labels']],submission_orders_valid[['session_type', 'labels']]])

In [ ]:
# COMPUTE METRIC
score = 0
weights = {'clicks': 0.10, 'carts': 0.30, 'orders': 0.60}
for t in ['clicks','carts','orders']:
    sub = submission_final_valid.loc[submission_final_valid.session_type.str.contains(t)].copy()
    sub['session'] = sub.session_type.apply(lambda x: int(x.split('_')[0]))
    sub.labels = sub.labels.apply(lambda x: [int(i) for i in x.split(' ')[:20]])
    test_labels = pd.read_parquet('test_cv_labels.parquet')
    test_labels = test_labels.loc[test_labels['type']==t]
    test_labels = test_labels.merge(sub, how='inner', on=['session'])
    test_labels['hits'] = test_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels))), axis=1)
    test_labels['gt_count'] = test_labels.ground_truth.str.len().clip(0,20)
    recall = test_labels['hits'].sum() / test_labels['gt_count'].sum()
    score += weights[t]*recall
    print(f'{t} recall =',recall)
    
print('=============')
print('Overall Recall =',score)
print('=============')

# Misc